<a href="https://colab.research.google.com/github/adamzki99/nlp-zlatan/blob/all-MiniLM-L6-v2-implementation/nlp_zlatan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/nlp-datasets/wizard_of_wikipedia

/content/drive/MyDrive/nlp-datasets/wizard_of_wikipedia


# all-MiniLM-L6-v2

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 

In [ ]:
import json

with open('valid_random_split.json', 'r') as file:
    json_data = file.read()
    data = json.loads(json_data)

print('Datatype:', type(data))

Datatype: <class 'list'>


In [ ]:
data_extract = []

for i, conversation in enumerate(data):

  data_extract.append("new_conv_" + str(i))

  for j, dialog in enumerate(conversation['dialog']):

    if "Wizard" in dialog['speaker']:

      if j == 0:
        continue

      data_extract.append({'wizard':dialog['text']})

    if "Apprentice" in dialog['speaker']:
      data_extract.append({'apprentice':dialog['text']})

data_extract[:10]

['new_conv_0',
 {'apprentice': "I like Gardening, even when I've only been doing it for a short time."},
 {'wizard': 'I live on a farm, we garden all year long, it is very relaxing.'},
 {'apprentice': "That sounds great.  I've always thought that I would love living in a farm, but I;ve always lived in the city.  What do you mostly plant?"},
 {'wizard': 'I have planted several fruits trees, tomatoes, jalepenos, bell peppers, onions, Garlic, and potatoes mostly.'},
 {'apprentice': 'Great, I love the idea of growing my own vegetables and fruits! Do you have animals in the farm?'},
 {'wizard': 'yes i do. Cows, chickens, Micro pigs, Guinneas, We also do forest growing also. we plants large pine trees.'},
 {'apprentice': 'Wow, it sounds amazing, the Micro-pigs are so cute! are they trainable to be well behaved?'},
 {'wizard': 'yes they are. they are extremely smart. most people buy them for inside pets. you can train the to use a litter box just like a cat. When families started using vines 

In [ ]:
pair = []
conversation_pairs = []

for i, text in enumerate(data_extract):

  if "new_conv_" in text:
    continue

  pair.append(text)

  if len(pair) == 2:
    
    entry = {'apprentice':"", 'wizard': ""}

    for _, e in enumerate(pair):

      if 'apprentice' in e.keys():
        entry['apprentice'] = e['apprentice']

      if 'wizard' in e.keys():
        entry['wizard'] = e['wizard']


    conversation_pairs.append(entry)
    pair = []

conversation_pairs[:5]

[{'apprentice': "I like Gardening, even when I've only been doing it for a short time.",
  'wizard': 'I live on a farm, we garden all year long, it is very relaxing.'},
 {'apprentice': "That sounds great.  I've always thought that I would love living in a farm, but I;ve always lived in the city.  What do you mostly plant?",
  'wizard': 'I have planted several fruits trees, tomatoes, jalepenos, bell peppers, onions, Garlic, and potatoes mostly.'},
 {'apprentice': 'Great, I love the idea of growing my own vegetables and fruits! Do you have animals in the farm?',
  'wizard': 'yes i do. Cows, chickens, Micro pigs, Guinneas, We also do forest growing also. we plants large pine trees.'},
 {'apprentice': 'Wow, it sounds amazing, the Micro-pigs are so cute! are they trainable to be well behaved?',
  'wizard': 'yes they are. they are extremely smart. most people buy them for inside pets. you can train the to use a litter box just like a cat. When families started using vines and trees to build 

In [ ]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
sentences = []

for i, pair in enumerate(conversation_pairs[:1000]):

  sentences.append([pair['apprentice'], pair['wizard']])


# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# Compute token embeddings
model.to(device)
encoded_input.to(device)

with torch.no_grad():
    model_output = model(**encoded_input)

## Using mean-pooling

We are using mean-pooling to aggregate the sentence embeddings into a vector representation that aims to capture the meaning of the sentence. This is performed by averaging the embeddings dimensions.

By also applying the attention mask wich is provided with the model, we are able improve the **accuracy** of the averaging. This is becase the function wont be taking into acount the words that wont provide usefull information, e.g. stop-words.

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

In [ ]:
%pip install hnswlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl size=2119666 sha256=a85d5ce17216427f749305835d865837d0de12b50ed9a01c5248d9f3cc05f187
  Stored in directory: /root/.cache/pip/wheels/8a/ae/ec/235a682e0041fbaeee389843670581ec6c66872db856dfa9a4
Successfully built hnswlib


In [ ]:
import hnswlib

# Create the HNSW index
index = hnswlib.Index(space='l2', dim=sentence_embeddings.shape[1])
index.init_index(max_elements=len(sentence_embeddings), ef_construction=200, M=16)

# Add sentence embeddings to the index
index.add_items(sentence_embeddings.cpu().numpy())

In [ ]:
message = []
message.append("I like Gardening, even when I've only been doing it for a short time.")

encoded_message = tokenizer(message, padding=True, truncation=True, return_tensors='pt')

encoded_message.to(device)

with torch.no_grad():
    model_output_message = model(**encoded_message)

# Perform pooling
query_embedding = mean_pooling(model_output_message, encoded_message['attention_mask'])

# Normalize embedding
#query_embedding = F.normalize(model_output_message, p=2, dim=1)

# Perform a similarity search
k = 5  # Number of closest neighbors to retrieve
labels, distances = index.knn_query(query_embedding.cpu(), k=k)

In [ ]:
for _, indexes in enumerate(labels):
  for i, ind in enumerate(indexes):
    print(sentences[ind][1], ".\t Distance:", distances[0][i])

I live on a farm, we garden all year long, it is very relaxing. .	 Distance: 8.451955
That's cool I grow sunflowers, oranges, lemons, aloe, a bunch other plants. I find gardening to be relaxing, do you? .	 Distance: 10.845751
I prefer to go to the beach and search for rocks that have been washed with sand to form little pebbles. .	 Distance: 11.277561
I bet gardening is exhausting definitely not something easy definitely a learning experience. .	 Distance: 11.392678
I have planted several fruits trees, tomatoes, jalepenos, bell peppers, onions, Garlic, and potatoes mostly. .	 Distance: 11.965621
